In [ ]:
import json
import time
import requests
import re
import random
from tqdm import tqdm
from collections import defaultdict

# ===== API Wrapper =====
class LLM_API_Wrapper:
    def __init__(self, model, api_key):
        self.model = model
        self.api_key = api_key

    def call_api(self, prompt_text: str, n: int = 1, timeout_s: int = 90):
        url = ""
        payload = {
            "model": self.model,
            "temperature": 0.6,                 # 略降温，提升结构化输出概率
            "n": n,
            "messages": [{"role": "user", "content": prompt_text}],
            "modalities": ["text"],
            "response_format": {"type": "text"},
            "max_completion_tokens": 512,
            "stream": False
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        try:
            resp = requests.post(url, headers=headers, json=payload, timeout=timeout_s)
            resp.raise_for_status()
            data = resp.json()
            if "choices" in data and data["choices"]:
                return [c["message"]["content"] for c in data["choices"]]
            print("⚠️ Missing 'choices' in response.")
            return []
        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")
            return []

# ===== Prompt Builders =====
def build_full_analysis_prompt(text: str) -> str:
    # 严格要求只返回 JSON
    return f"""
You are a psycholinguistics expert. Analyze the following social media post from three perspectives:

1) Semantic Summary: main idea or intention.
2) Sentiment Analysis: emotions/attitudes.
3) Linguistic Style: writing style (e.g., emotional, rational, informal, formal, vague).

Return ONLY valid JSON with the exact keys below and no extra text:

{{
  "semantic_view": "...",
  "sentiment_view": "...",
  "linguistic_view": "..."
}}

Post:
\"\"\"{(text or '').strip()[:1024]}\"\"\"
""".strip()

def build_fallback_prompt(text: str) -> str:
    # 退而求其次的更简单提示
    return f"""
Provide a STRICT JSON object with three short fields summarizing the post:

{{
  "semantic_view": "<1-2 sentences>",
  "sentiment_view": "<one or two emotions>",
  "linguistic_view": "<style words>"
}}

Post: {(text or '').strip()[:1024]}
""".strip()

# ===== JSON Parser =====
def safe_extract_json(raw: str) -> dict:
    """
    尽可能从多种常见返回格式中提取 JSON；不合法返回 None。
    """
    if not raw:
        return None
    txt = raw.strip()

    # 去掉代码块包裹
    if txt.startswith("```"):
        txt = re.sub(r"^```(json)?", "", txt, flags=re.IGNORECASE).strip()
        if txt.endswith("```"):
            txt = txt[:-3].strip()

    # 如果前后有冗余文字，尝试抽取第一个 {...} 块
    # 允许有换行/空白
    m = re.search(r"\{[\s\S]*\}", txt)
    if m:
        txt = m.group(0)

    # 单引号 -> 双引号（尽量不误伤）
    if txt.count('"') == 0 and txt.count("'") > 0:
        txt = txt.replace("’", "'").replace("‘", "'")
        txt = re.sub(r"(?<!\\)'", '"', txt)

    # 去掉尾部多余逗号
    txt = re.sub(r",\s*}", "}", txt)
    txt = re.sub(r",\s*]", "]", txt)

    try:
        obj = json.loads(txt)
        # 校验字段
        if not isinstance(obj, dict):
            return None
        for k in ("semantic_view", "sentiment_view", "linguistic_view"):
            if k not in obj or not isinstance(obj[k], str):
                return None
        return obj
    except Exception:
        return None

# ===== 兜底生成（确保每条都有）=====
def fallback_views_from_text(text: str) -> dict:
    """
    不依赖 LLM 的兜底：生成简短可用的三段字符串，确保字段不为空。
    """
    text = (text or "").strip()
    # 语义：截取前一句或前 120 字符
    semantic = ""
    if text:
        sentence = re.split(r"(?<=[.!?。！？])\s+", text)[0]
        semantic = sentence[:180]
    if not semantic:
        semantic = "The post contains limited context; the main idea is unclear."

    # 情感：关键词极简判断
    lowered = text.lower()
    if any(w in lowered for w in ["love", "great", "happy", "excited", "enjoy", "喜欢", "高兴", "开心"]):
        senti = "Positive, optimistic."
    elif any(w in lowered for w in ["hate", "angry", "sad", "tired", "annoyed", "讨厌", "生气", "难过", "疲惫"]):
        senti = "Negative, possibly frustrated or tired."
    else:
        senti = "Neutral or mixed."

    # 语言风格：根据表情/大小写/感叹号粗略判断
    style_tokens = []
    if re.search(r"[A-Z]{3,}", text): style_tokens.append("emphatic")
    if re.search(r"[!！]{1,}", text): style_tokens.append("expressive")
    if re.search(r":[)D]|😂|🤣|😅|🙂|😉", text): style_tokens.append("informal")
    if not style_tokens: style_tokens = ["conversational"]
    ling = ", ".join(style_tokens)

    return {
        "semantic_view": semantic,
        "sentiment_view": senti,
        "linguistic_view": ling
    }

# ===== 主流程设置 =====
MODEL_NAME = "gpt-4.1-mini"
API_KEY = ""               # <<< 用你自己的 key
MAX_PER_TYPE = 200
INPUT_FILE = "extended_mbti_dataset_v17.json"
OUTPUT_FILE = "mbti_sample_with_all_views.json"
MAX_RETRY = 5                                # 最多重试 5 次
BASE_SLEEP = 2                               # 初始退避秒

# ===== Init =====
llm = LLM_API_Wrapper(model=MODEL_NAME, api_key=API_KEY)

# ===== Load data =====
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    full_data = json.load(f)

# ===== Balance by type =====
type_counter = defaultdict(int)
selected_samples = []
for item in full_data:
    mbti = item["type"]
    if MAX_PER_TYPE is None or type_counter[mbti] < MAX_PER_TYPE:
        selected_samples.append(item.copy())
        type_counter[mbti] += 1
    if MAX_PER_TYPE and len(selected_samples) >= 16 * MAX_PER_TYPE:
        break

print(f"✅ Selected {len(selected_samples)} samples ({MAX_PER_TYPE} per MBTI type)")

# ===== Process =====
failed_cases = []
for i, item in enumerate(tqdm(selected_samples, desc="Generating views")):
    post = item.get("posts") or item.get("posts_cleaned") or ""
    prompt = build_full_analysis_prompt(post)

    views = None
    for attempt in range(1, MAX_RETRY + 1):
        result = llm.call_api(prompt)
        parsed = safe_extract_json(result[0]) if result else None

        if parsed:
            # 校验非空
            if all(isinstance(parsed[k], str) and parsed[k].strip() for k in parsed):
                views = parsed
                break
            else:
                # 返回了 JSON 但内容空，换更简单的 backup prompt
                prompt = build_fallback_prompt(post)
        else:
            # 第一次失败后改用 fallback prompt，提高成功率
            prompt = build_fallback_prompt(post)

        # 指数退避 + 抖动
        sleep_s = BASE_SLEEP * attempt + random.uniform(0, 0.5 * attempt)
        print(f"⚠️ Bad/invalid JSON (attempt {attempt}), retry in {sleep_s:.1f}s ...")
        time.sleep(sleep_s)

    # 如果还是失败，使用兜底生成（确保每条都有）
    if not views:
        print("❗ Using local fallback generator.")
        views = fallback_views_from_text(post)
        failed_cases.append({"type": item.get("type"), "posts": post})

    item["semantic_view"] = views.get("semantic_view", "")
    item["sentiment_view"] = views.get("sentiment_view", "")
    item["linguistic_view"] = views.get("linguistic_view", "")

    # 可选：定期落盘，支持断点续跑
    if (i + 1) % 20 == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(selected_samples, f, ensure_ascii=False, indent=2)

# ===== Save =====
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(selected_samples, f, ensure_ascii=False, indent=2)

print(f"\n✅ Done! Saved enriched samples to {OUTPUT_FILE}")


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import json
from torch.nn import functional as F
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch_optimizer as optim 

In [ ]:
def mbti_to_binary(mbti_type):
    # INFP → [0, 1, 0, 1] (E/I, S/N, T/F, J/P)
    return {
        "ei": 0 if mbti_type[0] == "I" else 1,
        "ns": 0 if mbti_type[1] == "S" else 1,
        "tf": 0 if mbti_type[2] == "F" else 1,
        "jp": 0 if mbti_type[3] == "P" else 1
    }
class JointMBTIDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # 基础文本
        base_text = item.get("posts_cleaned", item["posts"]) or ""

        # 取三视角（如果没有就空字符串）
        semantic = item.get("semantic_view", "")
        sentiment = item.get("sentiment_view", "")
        linguistic = item.get("linguistic_view", "")

        # 拼接：原文 + 三视角
        combined_text = base_text
        if any([semantic, sentiment, linguistic]):
            combined_text = f"{base_text} [SEP] {semantic} [SEP] {sentiment} [SEP] {linguistic}"

        mbti_type = item["type"]
        mbti_label = mbti_to_binary(mbti_type)

        encoding = self.tokenizer(
            combined_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "ei": torch.tensor(mbti_label["ei"], dtype=torch.float),
            "ns": torch.tensor(mbti_label["ns"], dtype=torch.float),
            "tf": torch.tensor(mbti_label["tf"], dtype=torch.float),
            "jp": torch.tensor(mbti_label["jp"], dtype=torch.float),
            "label": mbti_type
        }
class JointMBTIModel(nn.Module):
    def __init__(self, encoder_name="microsoft/deberta-v3-base", dropout=0.1):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name)
        hidden_size = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        self.classifier_ei = nn.Linear(hidden_size, 1)
        self.classifier_ns = nn.Linear(hidden_size, 1)
        self.classifier_tf = nn.Linear(hidden_size, 1)
        self.classifier_jp = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        output = self.encoder(input_ids, attention_mask=attention_mask)
        cls_emb = self.dropout(output.last_hidden_state[:, 0, :])
        return {
            "embedding": cls_emb,  # 可导出向量
            "ei": self.classifier_ei(cls_emb),
            "ns": self.classifier_ns(cls_emb),
            "tf": self.classifier_tf(cls_emb),
            "jp": self.classifier_jp(cls_emb)
        }
def compute_mbti_loss(preds, labels):
    bce = nn.BCEWithLogitsLoss()
    return (
        bce(preds["ei"].squeeze(), labels["ei"]) +
        bce(preds["ns"].squeeze(), labels["ns"]) +
        bce(preds["tf"].squeeze(), labels["tf"]) +
        bce(preds["jp"].squeeze(), labels["jp"])
    ) / 4
    
def train_model(model, train_loader, val_loader, optimizer, device, epochs=5, save_path="kaggle_best_fem.pt"):
    model.to(device)
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{epochs}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = {k: batch[k].to(device) for k in ["ei", "ns", "tf", "jp"]}

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = compute_mbti_loss(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"\nEpoch {epoch+1} - Train Loss: {avg_loss:.4f}")

        # 验证阶段
        model.eval()
        correct_ei = correct_ns = correct_tf = correct_jp = correct_all = 0
        total = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"[Val]   Epoch {epoch+1}"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = {k: batch[k].to(device) for k in ["ei", "ns", "tf", "jp"]}

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)

                pred_ei = torch.sigmoid(outputs["ei"]).round()
                pred_ns = torch.sigmoid(outputs["ns"]).round()
                pred_tf = torch.sigmoid(outputs["tf"]).round()
                pred_jp = torch.sigmoid(outputs["jp"]).round()

                correct_ei += (pred_ei.squeeze() == labels["ei"]).sum().item()
                correct_ns += (pred_ns.squeeze() == labels["ns"]).sum().item()
                correct_tf += (pred_tf.squeeze() == labels["tf"]).sum().item()
                correct_jp += (pred_jp.squeeze() == labels["jp"]).sum().item()

                correct_all += (
                    (pred_ei.squeeze() == labels["ei"]) &
                    (pred_ns.squeeze() == labels["ns"]) &
                    (pred_tf.squeeze() == labels["tf"]) &
                    (pred_jp.squeeze() == labels["jp"])
                ).sum().item()

                total += input_ids.size(0)

        acc_ei = correct_ei / total
        acc_ns = correct_ns / total
        acc_tf = correct_tf / total
        acc_jp = correct_jp / total
        acc_all = correct_all / total

        print(f"Validation Accuracy:")
        print(f"  EI: {acc_ei:.2%} | NS: {acc_ns:.2%} | TF: {acc_tf:.2%} | JP: {acc_jp:.2%}")
        print(f"  4D Match: {acc_all:.2%}")

        # 保存验证集最佳模型
        if acc_all > best_val_acc:
            best_val_acc = acc_all
            torch.save(model.state_dict(), save_path)
            print(f"✅ Best model saved to {save_path} (4D match: {acc_all:.2%})")
def visualize_embeddings(model, dataloader, device):
    model.eval()
    all_embeds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            all_embeds.append(output["embedding"].cpu())
            all_labels.extend(batch["label"])
    embeddings = torch.cat(all_embeds).numpy()

    tsne = TSNE(n_components=2, random_state=42).fit_transform(embeddings)
    df = pd.DataFrame(tsne, columns=["x", "y"])
    df["label"] = all_labels

    plt.figure(figsize=(10, 8))
    for label in sorted(set(all_labels)):
        subset = df[df["label"] == label]
        plt.scatter(subset["x"], subset["y"], label=label, alpha=0.6)
    plt.legend()
    plt.title("FEM Personality Embedding Space (t-SNE)")
    plt.show()


In [ ]:
if __name__ == "__main__":
    # 加载数据
    with open("mbti_sample_with_all_views.json", "r", encoding="utf-8") as f:
        full_data = json.load(f)

    # 拆分训练集 / 验证集
    train_data, val_data = train_test_split(full_data, test_size=0.2, random_state=42)

    # 初始化 tokenizer
    tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

    # 初始化 dataset
    train_dataset = JointMBTIDataset(train_data, tokenizer)  # 支持list
    val_dataset = JointMBTIDataset(val_data, tokenizer)

    # 初始化 dataloader
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)

    # 模型和优化器
    model = JointMBTIModel("microsoft/deberta-v3-large")
    #optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    optimizer = optim.AdamP(model.parameters(), lr=1e-5, weight_decay=0.01)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 训练并保存最佳模型
    train_model(model, train_loader, val_loader, optimizer, device, epochs=15)

    # 可视化嵌入空间（只用验证集）
    visualize_embeddings(model, val_loader, device)